## Problem 1: Geocode shopping centers (5 points)

The overall aim of problems 1-3 is to find out **how many people live within a walking distance (1.5 km) from certain shopping centers in Helsinki**.

In problem 1 aim is to find out the addresses of shopping centers and to retreive numercal coordinates for these addresses. As an output, we will have a Shapefile called `shopping_centers.shp` which contains the geocoded result.

**Preparation:** Find out the addresses for following shopping centers from the internet, and write the addresses into a text file called `shopping_centers.txt`:

 - Itis
 - Forum
 - Iso-omena
 - Sello
 - Jumbo
 - REDI
 - Tripla 
 
 *Hint for "Tripla": This shopping center opened in October 10 2019, and if you are doing this exercise soon after that, the official address might not yet be in online databases. 
 Check for an address nearby the Pasila railway station on OpenStreetMap.org and use that as input.*

`shopping_centers.txt` should have semicolon (`;`) as a separator, and the file should include the following columns:

- ``id`` (integer) containing an unique identifier for each shopping center
- ``name`` (string) of each shopping center
- ``addr`` (string) the address 


See and example of how to format the text file [in the lesson 3 materials](https://automating-gis-processes.github.io/site/master/notebooks/L3/geocoding_in_geopandas.html). Save (and upload) the text file into your exercise repository.

- Read `shopping_centers.txt` that you just created into a pandas DataFrame called ``data``:

In [1]:
# Import modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Read the data (replace "None" with your own code)
fp= r'C:\Users\paul-\Desktop\AutoGIS\Lesson3\Exercise\data\shopping_centers.txt'

data = pd.read_csv(fp, sep=';')

# YOUR CODE HERE
try:
    print(data.head())
except:
    raise NotImplementedError()

     id       name                                        addr
0  1000       Itis          Itäkatu 7, 00930 Helsinki, Finland
1  1001     Center        Kaivokatu 1, 00100 Helsinki, Finland
2  1002  Iso-omena    Piispansilta 11, 02230 Helsinki, Finland
3  1003      Sello  Leppävaarankatu 3, 02600 Helsinki, Finland
4  1004      Jumbo     Valuuttakatu 3, 01510 Helsinki, Finland


In [2]:
#NON-EDITABLE TEST CELL
# Check your input data
print(data)

     id       name                                        addr
0  1000       Itis          Itäkatu 7, 00930 Helsinki, Finland
1  1001     Center        Kaivokatu 1, 00100 Helsinki, Finland
2  1002  Iso-omena    Piispansilta 11, 02230 Helsinki, Finland
3  1003      Sello  Leppävaarankatu 3, 02600 Helsinki, Finland
4  1004      Jumbo     Valuuttakatu 3, 01510 Helsinki, Finland


- Geocode the addresses using the Nominatim geocoding service. Store the output in a variable called `geo`:

In [3]:
# Geocode the addresses using Nominatim
from geopandas.tools import geocode

geo = geocode(data['addr'], provider='Nominatim', user_agent='autogis_PH', timeout=4)
geo.head()


,geometry,address
0,POINT (25.08294 60.21170),"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va..."
1,POINT (24.94135 60.17178),"Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel..."
2,POINT (24.73779 60.16294),"Sports Academy, 11, Piispansilta, Nuottaniemi,..."
3,POINT (24.81433 60.21770),"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete..."
4,POINT (24.96486 60.29219),"Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi..."


In [4]:
#NON-EDITABLE TEST CELL
# Check the geocoded output
print(geo)

                    geometry  \
0  POINT (25.08294 60.21170)   
1  POINT (24.94135 60.17178)   
2  POINT (24.73779 60.16294)   
3  POINT (24.81433 60.21770)   
4  POINT (24.96486 60.29219)   

                                             address  
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...  
1  Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...  
2  Sports Academy, 11, Piispansilta, Nuottaniemi,...  
3  Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...  
4  Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi...  


In [5]:
#NON-EDITABLE TEST CELL
# Check the data type (should be a GeoDataFrame!)
print(type(geo))

<class 'geopandas.geodataframe.GeoDataFrame'>


Check that the coordinate reference system of the geocoded result is correctly defined, and **reproject the layer into ETRS GK-25** (EPSG:3879):

In [6]:
# YOUR CODE HERE
geo_epsg4326 = geo

In [7]:
geo = geo.to_crs(epsg=3879)
geo.head()

,geometry,address
0,POINT (25504598.602 6677662.109),"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va..."
1,POINT (25496744.578 6673212.851),"Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel..."
2,POINT (25485440.532 6672255.563),"Sports Academy, 11, Piispansilta, Nuottaniemi,..."
3,POINT (25489707.583 6678342.162),"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete..."
4,POINT (25498056.327 6686627.539),"Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi..."


In [8]:
#NON-EDITABLE TEST CELL
# Check layer crs
print(geo.crs)

epsg:3879


- Make a table join between the geocoded addresses (``geo``) and the original addresses (``data``) in order to link the numerical coordinates and  the `id` and `name` of each shopping center. 
- Store the output in a variable called ``geodata`` 


In [9]:
# Join the tables
geodata = geo.join(data)
geodata = geodata[['id','name','address','addr','geometry']]
geodata.head()

,id,name,address,addr,geometry
0,1000,Itis,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","Itäkatu 7, 00930 Helsinki, Finland",POINT (25504598.602 6677662.109)
1,1001,Center,"Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...","Kaivokatu 1, 00100 Helsinki, Finland",POINT (25496744.578 6673212.851)
2,1002,Iso-omena,"Sports Academy, 11, Piispansilta, Nuottaniemi,...","Piispansilta 11, 02230 Helsinki, Finland",POINT (25485440.532 6672255.563)
3,1003,Sello,"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...","Leppävaarankatu 3, 02600 Helsinki, Finland",POINT (25489707.583 6678342.162)
4,1004,Jumbo,"Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi...","Valuuttakatu 3, 01510 Helsinki, Finland",POINT (25498056.327 6686627.539)


In [10]:
#NON-EDITABLE TEST CELL
# Check the join output
print(geodata.head())

     id       name                                            address  \
0  1000       Itis  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...   
1  1001     Center  Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...   
2  1002  Iso-omena  Sports Academy, 11, Piispansilta, Nuottaniemi,...   
3  1003      Sello  Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...   
4  1004      Jumbo  Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi...   

                                         addr  \
0          Itäkatu 7, 00930 Helsinki, Finland   
1        Kaivokatu 1, 00100 Helsinki, Finland   
2    Piispansilta 11, 02230 Helsinki, Finland   
3  Leppävaarankatu 3, 02600 Helsinki, Finland   
4     Valuuttakatu 3, 01510 Helsinki, Finland   

                           geometry  
0  POINT (25504598.602 6677662.109)  
1  POINT (25496744.578 6673212.851)  
2  POINT (25485440.532 6672255.563)  
3  POINT (25489707.583 6678342.162)  
4  POINT (25498056.327 6686627.539)  


- Save the output as a Shapefile called `shopping_centers.shp` 

In [11]:
# Define output filepath
out_fp = r'C:\Users\paul-\Desktop\AutoGIS\Lesson3\Exercise\data\shopping_centers.shp'

# Save file
geodata.to_file(out_fp)


In [12]:
#NON-EDITABLE TEST CELL
# Print info about output file
print("Geocoded output is stored in this file:", out_fp)

Geocoded output is stored in this file: C:\Users\paul-\Desktop\AutoGIS\Lesson3\Exercise\data\shopping_centers.shp


## Problem 2: Create buffers around shopping centers (5 points)

Let's continue with our case study and calculate a 1.5 km buffer around the geocoded points. 


- Start by creating a new column called `buffer` to ``geodata`` GeoDataFrame:

In [13]:
# YOUR CODE HERE
geodata['buffer'] = None

geodata.head()


,id,name,address,addr,geometry,buffer
0,1000,Itis,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","Itäkatu 7, 00930 Helsinki, Finland",POINT (25504598.602 6677662.109),None
1,1001,Center,"Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...","Kaivokatu 1, 00100 Helsinki, Finland",POINT (25496744.578 6673212.851),None
2,1002,Iso-omena,"Sports Academy, 11, Piispansilta, Nuottaniemi,...","Piispansilta 11, 02230 Helsinki, Finland",POINT (25485440.532 6672255.563),None
3,1003,Sello,"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...","Leppävaarankatu 3, 02600 Helsinki, Finland",POINT (25489707.583 6678342.162),None
4,1004,Jumbo,"Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi...","Valuuttakatu 3, 01510 Helsinki, Finland",POINT (25498056.327 6686627.539),None


- Calculate a 1.5 km buffer for each geocoded point. Store the buffer geometry in the new `buffer` column.

Here, you can use the [GeoDataFrame buffer() method](http://geopandas.org/geometric_manipulations.html#GeoSeries.buffer), which uses Shapely's [buffer](http://toblerity.org/shapely/manual.html#object.buffer) in the bacground. You only need to use the `distance` -parameter, don't worry about the other parameters.

In [14]:
# YOUR CODE HERE
geodata['buffer'] = geodata.buffer(distance=1500)
geodata.head()

,id,name,address,addr,geometry,buffer
0,1000,Itis,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","Itäkatu 7, 00930 Helsinki, Finland",POINT (25504598.602 6677662.109),"POLYGON ((25506098.602 6677662.109, 25506091.3..."
1,1001,Center,"Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...","Kaivokatu 1, 00100 Helsinki, Finland",POINT (25496744.578 6673212.851),"POLYGON ((25498244.578 6673212.851, 25498237.3..."
2,1002,Iso-omena,"Sports Academy, 11, Piispansilta, Nuottaniemi,...","Piispansilta 11, 02230 Helsinki, Finland",POINT (25485440.532 6672255.563),"POLYGON ((25486940.532 6672255.563, 25486933.3..."
3,1003,Sello,"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...","Leppävaarankatu 3, 02600 Helsinki, Finland",POINT (25489707.583 6678342.162),"POLYGON ((25491207.583 6678342.162, 25491200.3..."
4,1004,Jumbo,"Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi...","Valuuttakatu 3, 01510 Helsinki, Finland",POINT (25498056.327 6686627.539),"POLYGON ((25499556.327 6686627.539, 25499549.1..."


In [15]:
#NON-EDITABLE TEST CELL
print(geodata.head())

     id       name                                            address  \
0  1000       Itis  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...   
1  1001     Center  Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...   
2  1002  Iso-omena  Sports Academy, 11, Piispansilta, Nuottaniemi,...   
3  1003      Sello  Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...   
4  1004      Jumbo  Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi...   

                                         addr  \
0          Itäkatu 7, 00930 Helsinki, Finland   
1        Kaivokatu 1, 00100 Helsinki, Finland   
2    Piispansilta 11, 02230 Helsinki, Finland   
3  Leppävaarankatu 3, 02600 Helsinki, Finland   
4     Valuuttakatu 3, 01510 Helsinki, Finland   

                           geometry  \
0  POINT (25504598.602 6677662.109)   
1  POINT (25496744.578 6673212.851)   
2  POINT (25485440.532 6672255.563)   
3  POINT (25489707.583 6678342.162)   
4  POINT (25498056.327 6686627.539)   

                               

In [16]:
#NON-EDITABLE TEST CELL
# Check the data type of the first value in the buffer-column
print(type(geodata.at[0,'buffer']))

<class 'shapely.geometry.polygon.Polygon'>


In [17]:
#NON-EDITABLE TEST CELL
# Check the areas of your buffers in km^2
print(round(gpd.GeoSeries(geodata["buffer"]).area / 1000000))

0    7.0
1    7.0
2    7.0
3    7.0
4    7.0
dtype: float64


- Replace the values in `geometry` column with the values of `buffer` column:

In [18]:
geodata.head()

,id,name,address,addr,geometry,buffer
0,1000,Itis,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","Itäkatu 7, 00930 Helsinki, Finland",POINT (25504598.602 6677662.109),"POLYGON ((25506098.602 6677662.109, 25506091.3..."
1,1001,Center,"Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...","Kaivokatu 1, 00100 Helsinki, Finland",POINT (25496744.578 6673212.851),"POLYGON ((25498244.578 6673212.851, 25498237.3..."
2,1002,Iso-omena,"Sports Academy, 11, Piispansilta, Nuottaniemi,...","Piispansilta 11, 02230 Helsinki, Finland",POINT (25485440.532 6672255.563),"POLYGON ((25486940.532 6672255.563, 25486933.3..."
3,1003,Sello,"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...","Leppävaarankatu 3, 02600 Helsinki, Finland",POINT (25489707.583 6678342.162),"POLYGON ((25491207.583 6678342.162, 25491200.3..."
4,1004,Jumbo,"Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi...","Valuuttakatu 3, 01510 Helsinki, Finland",POINT (25498056.327 6686627.539),"POLYGON ((25499556.327 6686627.539, 25499549.1..."


In [19]:
# YOUR CODE HERE
geodata['geometry'] = geodata['buffer']


In [20]:
#NON-EDITABLE TEST CELL
print(geodata.head())

     id       name                                            address  \
0  1000       Itis  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...   
1  1001     Center  Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...   
2  1002  Iso-omena  Sports Academy, 11, Piispansilta, Nuottaniemi,...   
3  1003      Sello  Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...   
4  1004      Jumbo  Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi...   

                                         addr  \
0          Itäkatu 7, 00930 Helsinki, Finland   
1        Kaivokatu 1, 00100 Helsinki, Finland   
2    Piispansilta 11, 02230 Helsinki, Finland   
3  Leppävaarankatu 3, 02600 Helsinki, Finland   
4     Valuuttakatu 3, 01510 Helsinki, Finland   

                                            geometry  \
0  POLYGON ((25506098.602 6677662.109, 25506091.3...   
1  POLYGON ((25498244.578 6673212.851, 25498237.3...   
2  POLYGON ((25486940.532 6672255.563, 25486933.3...   
3  POLYGON ((25491207.583 6678342.162, 254

In [47]:
len(geodata)

5

Optional: at this point, you can drop out unnecessary columns from the geodataframe. In the next problem, 
we will only need these columns: `'id', 'name', 'geometry'`

In [21]:
geodata.head()

,id,name,address,addr,geometry,buffer
0,1000,Itis,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","Itäkatu 7, 00930 Helsinki, Finland","POLYGON ((25506098.602 6677662.109, 25506091.3...","POLYGON ((25506098.602 6677662.109, 25506091.3..."
1,1001,Center,"Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...","Kaivokatu 1, 00100 Helsinki, Finland","POLYGON ((25498244.578 6673212.851, 25498237.3...","POLYGON ((25498244.578 6673212.851, 25498237.3..."
2,1002,Iso-omena,"Sports Academy, 11, Piispansilta, Nuottaniemi,...","Piispansilta 11, 02230 Helsinki, Finland","POLYGON ((25486940.532 6672255.563, 25486933.3...","POLYGON ((25486940.532 6672255.563, 25486933.3..."
3,1003,Sello,"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...","Leppävaarankatu 3, 02600 Helsinki, Finland","POLYGON ((25491207.583 6678342.162, 25491200.3...","POLYGON ((25491207.583 6678342.162, 25491200.3..."
4,1004,Jumbo,"Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi...","Valuuttakatu 3, 01510 Helsinki, Finland","POLYGON ((25499556.327 6686627.539, 25499549.1...","POLYGON ((25499556.327 6686627.539, 25499549.1..."


In [48]:
geodata.drop(['address','addr','buffer'], axis=1, inplace=True)

## Problem 3: How many people live near shopping centers? (5 points)

Last step in our analysis is to make a spatial join between our buffer layer and population data in order to find out **how many people live near each shopping center**. We will use the same data as we did during [lesson 3](https://automating-gis-processes.github.io/site/notebooks/L3/spatial-join.html#Spatial-join); **a Population Grid** that is available via the HSY wfs. 

Alternatively, you can also download the data from the [Helsinki Region Infoshare (HRI)](https://www.hsy.fi/fi/asiantuntijalle/avoindata/Sivut/AvoinData.aspx?dataID=7) as a shapefile (using wget).

The coordinate reference system of the population grid is **ETRS GK-25 (EPSG:3879)**.


**Steps:**

- Read the population grid into a geodataframe

- Select only the useful columns from the population grid: ``'asukkaita'`` (=population count per grid square) and ``'geometry'`` 

- Make a spatial join between your buffered point layer and population grid layer. Join the information now from buffer layer **into the population grid layer**

- Group the joined layer by shopping center index

- Calculate the sum of population living within 1.5 km for each shopping center.

**Finally:**

- Print out the population living within 1.5 km from each shopping center:

     - Itis
     - Forum
     - Iso-omena
     - Sello
     - Jumbo
     - REDI
     - Tripla
     
**Final print out should contain both the shopping center name and population count**, for example: `25858 people live within 1.5 km from Iso-Omena`.

*Feel free to divide your solution into several codeblocks! Remember to comment your code  :)*

In [49]:
import geopandas as gpd

fp = r'C:\Users\paul-\Desktop\AutoGIS\Lesson3\Exercise\data\Vaestotietoruudukko_2019.shp'

pop = gpd.read_file(fp)

In [50]:
pop.head()

,INDEX,ASUKKAITA,ASVALJYYS,IKA0_9,IKA10_19,IKA20_29,IKA30_39,IKA40_49,IKA50_59,IKA60_69,IKA70_79,IKA_YLI80,geometry
0,703,5,51.0,99,99,99,99,99,99,99,99,99,"POLYGON Z ((25472499.995 6685998.998 0.000, 25..."
1,710,8,44.0,99,99,99,99,99,99,99,99,99,"POLYGON Z ((25472499.995 6684249.004 0.000, 25..."
2,711,5,90.0,99,99,99,99,99,99,99,99,99,"POLYGON Z ((25472499.995 6683999.005 0.000, 25..."
3,715,12,37.0,99,99,99,99,99,99,99,99,99,"POLYGON Z ((25472499.995 6682998.998 0.000, 25..."
4,848,5,53.0,99,99,99,99,99,99,99,99,99,"POLYGON Z ((25472749.993 6690249.003 0.000, 25..."


In [51]:
pop = pop.rename(columns={'ASUKKAITA': 'POP18'})


In [111]:
pop.head()

,POP18,geometry
0,5,"POLYGON Z ((25472499.995 6685998.998 0.000, 25..."
1,8,"POLYGON Z ((25472499.995 6684249.004 0.000, 25..."
2,5,"POLYGON Z ((25472499.995 6683999.005 0.000, 25..."
3,12,"POLYGON Z ((25472499.995 6682998.998 0.000, 25..."
4,5,"POLYGON Z ((25472749.993 6690249.003 0.000, 25..."


In [112]:
pop = pop[['POP18', 'geometry']]
pop.head()

,POP18,geometry
0,5,"POLYGON Z ((25472499.995 6685998.998 0.000, 25..."
1,8,"POLYGON Z ((25472499.995 6684249.004 0.000, 25..."
2,5,"POLYGON Z ((25472499.995 6683999.005 0.000, 25..."
3,12,"POLYGON Z ((25472499.995 6682998.998 0.000, 25..."
4,5,"POLYGON Z ((25472749.993 6690249.003 0.000, 25..."


In [113]:
pop.crs == geodata.crs

True

In [114]:
pop.crs

<Projected CRS: PROJCS["ETRS89_ETRS_GK25FIN_2010",GEOGCS["ETRS89", ...>
Name: ETRS89_ETRS_GK25FIN_2010
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: Geodetic_Reference_System_of_1980
- Prime Meridian: Greenwich

In [115]:
geodata.crs

<Projected CRS: PROJCS["ETRS89_ETRS_GK25FIN_2010",GEOGCS["ETRS89", ...>
Name: ETRS89_ETRS_GK25FIN_2010
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: Geodetic_Reference_System_of_1980
- Prime Meridian: Greenwich

In [116]:
geodata = geodata.to_crs(pop.crs)

In [117]:
geodata.crs

<Projected CRS: PROJCS["ETRS89_ETRS_GK25FIN_2010",GEOGCS["ETRS89", ...>
Name: ETRS89_ETRS_GK25FIN_2010
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: Geodetic_Reference_System_of_1980
- Prime Meridian: Greenwich

In [118]:
geodata.crs == pop.crs

True

In [119]:
#NON-EDITABLE TEST CELL
# Check your input data
print("Number of rows:", len(pop))
print(pop.head(3))

Number of rows: 5835
   POP18                                           geometry
0      5  POLYGON Z ((25472499.995 6685998.998 0.000, 25...
1      8  POLYGON Z ((25472499.995 6684249.004 0.000, 25...
2      5  POLYGON Z ((25472499.995 6683999.005 0.000, 25...


In [123]:
# Create a spatial join between grid layer and buffer layer. 
join_layer = gpd.sjoin(geodata, pop, how='left', op='contains')

In [124]:
join_layer.head()

,id,name,geometry,index_right,POP18
0,1000,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3...",4922,679
0,1000,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3...",5105,88
0,1000,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3...",5165,145
0,1000,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3...",5104,199
0,1000,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3...",5164,220


In [129]:
grouped = join_layer.groupby(by='id')

In [131]:
len(grouped)

5

In [152]:
# Report how many people live within 1.5 km distance from each shopping center

for name, group in grouped:
    sumG = sum(group['POP18'])
    #name = key
    print(sumG,'people live within 1.5 km from', name)

21015 people live within 1.5 km from 1000
55752 people live within 1.5 km from 1001
27500 people live within 1.5 km from 1002
23620 people live within 1.5 km from 1003
10970 people live within 1.5 km from 1004


**Reflections:**
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

YOUR ANSWER HERE

Well done! Now you can continue to [problem 4](Exercise-3-Problem-4.ipynb)